In [23]:
import pandas as pd
import pickle
with open('./X_data.pkl','rb') as file:
    X_data=pickle.load(file)
with open('./Y_data.pkl','rb') as file:
    Y_data=pickle.load(file)
with open('./X_test.pkl','rb') as file:
    X_test=pickle.load(file)

In [24]:
X_data[['regDate','creatDate']]

,regDate,creatDate
0,20040402,20160404
1,20030301,20160309
2,20040403,20160402
3,19960908,20160312
4,20120103,20160313
...,...,...
149995,20000607,20160327
149996,20091102,20160312
149997,20101003,20160328
149998,20060312,20160401


In [25]:
# 对日期格式进行清洗
def f(x):
    x=str(x)
    if x[4:6]=='00':
        return x[0:4]+'-'+'01'+'-'+x[6:8]
    else:
        return x[0:4]+'-'+x[4:6]+'-'+x[6:8]
    
X_data['regDate']=pd.to_datetime(X_data['regDate'].apply(f)) #这里apply运用了一个自己写的函数，来调整时间格式
X_data['creatDate']=pd.to_datetime(X_data['creatDate'].apply(f))
X_data[['regDate','creatDate']]        

,regDate,creatDate
0,2004-04-02,2016-04-04
1,2003-03-01,2016-03-09
2,2004-04-03,2016-04-02
3,1996-09-08,2016-03-12
4,2012-01-03,2016-03-13
...,...,...
149995,2000-06-07,2016-03-27
149996,2009-11-02,2016-03-12
149997,2010-10-03,2016-03-28
149998,2006-03-12,2016-04-01


In [31]:
# 对X_test数据也要处理一下
X_test['regDate']=pd.to_datetime(X_test['regDate'].apply(f)) #这里apply运用了一个自己写的函数，来调整时间格式
X_test['creatDate']=pd.to_datetime(X_test['creatDate'].apply(f))

In [32]:
#提取时间多尺度
X_data['regDate_y']=X_data['regDate'].dt.year
X_data['regDate_m']=X_data['regDate'].dt.month
X_data['regDate_d']=X_data['regDate'].dt.day
X_data

X_test['regDate_y']=X_test['regDate'].dt.year
X_test['regDate_m']=X_test['regDate'].dt.month
X_test['regDate_d']=X_test['regDate'].dt.day

In [39]:
#提取时间多尺度
X_data['creatDate_y']=X_data['creatDate'].dt.year
X_data['creatDate_m']=X_data['creatDate'].dt.month
X_data['creatDate_d']=X_data['creatDate'].dt.day


X_test['creatDate_y']=X_test['creatDate'].dt.year
X_test['creatDate_m']=X_test['creatDate'].dt.month
X_test['creatDate_d']=X_test['creatDate'].dt.day

In [40]:
#提取时间diff
#1991-01-01
X_data['regDate'].min()
X_test['regDate'].min()
base_date=pd.to_datetime('1991-01-01')
X_data['regDate_diff']=(X_data['regDate']-base_date).dt.days #间隔的天数
X_test['regDate_diff']=(X_test['regDate']-base_date).dt.days

In [54]:
#提取时间diff
#1991-01-01
X_data['creatDate'].min()
X_test['creatDate'].min()
base_date=pd.to_datetime('2014-03-10')# 这里应该找训练集，测试集中最小的数据

X_data['creatDate_diff']=(X_data['creatDate']-base_date).dt.days #间隔的天数
X_test['creatDate_diff']=(X_test['creatDate']-base_date).dt.days


In [55]:
#构造新特征'creatDate-regDate
X_data['creatDate-regDate']=(X_data['creatDate']-X_data['regDate']).dt.days #注意（）括号
X_test['creatDate-regDate']=(X_test['creatDate']-X_test['regDate']).dt.days #注意（）括号

In [57]:
#去掉原始的日期类型
X_data.drop(['regDate','creatDate'],axis=1,inplace=True)
X_test.drop(['regDate','creatDate'],axis=1,inplace=True)


In [58]:
## 数据集切分
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(X_data,Y_data,test_size=0.2,random_state=2021)
import xgboost as xgb
#创建xgb模型
xgb=xgb.XGBRegressor(max_depth=6, learning_rate=0.01,n_estimators=2000,
                     objective='reg:linear',tree_method='gpu_hist',
                     subsample=0.8,colsample_bytree=0.8,
                    min_child_samples=3, eval_metric='auc',reg_lambda=0.5)
xgb.fit(x_train,y_train)
y_pred=xgb.predict(x_val)

[17:10:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "min_child_samples" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [61]:
import numpy as np
#统计price的分布
def show_stats(data):
    print('min',np.min(data))
    print('max',np.max(data))
    print('ptp',np.ptp(data)) #计算最大值与最小值的差(极差) peak to peak
    print('mean',np.mean(data))
    print('std',np.std(data)) #标准差
    print('median',np.median(data))
#查询price的统计
show_stats(y_pred)


min -3187.6704
max 87733.72
ptp 90921.39
mean 5914.145
std 7323.731
median 3308.695


In [62]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred,y_val)

590.9323803388496

In [63]:
#重新训练模型,使用全量数据
#创建xgb模型
xgb.fit(X_data,Y_data)
y_pred=xgb.predict(X_test)
show_stats(y_pred)

[17:14:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "min_child_samples" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


min -808.59863
max 90722.86
ptp 91531.46
mean 5908.229
std 7346.697
median 3264.6226


In [20]:
#输出结果

result=pd.DataFrame(columns=['SaleID','price'])
test_data=pd.read_csv('used_car_testB_20200421.csv',sep=' ')

result['SaleID']=test_data['SaleID']
result['price']=y_pred

result.loc[result['price']<11,'price']=11 #将结果中price<11的部分重新定义为11
result.to_csv('./xgb_baseline2.csv',index=False)

## 使用XGBoost,对日期特征进行了处理
* Score=585.2867